In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from pathlib import Path

In `config`:

    config = Config(
        max_length = [512, 512]  # in tokens, list for title vs abstract
        from_pretrained = "facebook/bart-base"
        model_type = "Twin"
    )

In [4]:
from smtag.config import config
assert config.max_length==[512, 512]
assert config.from_pretrained=='facebook/bart-base'
assert config.model_type=='Twin'

In [5]:
config

Config(max_length=[512, 512], truncation=True, min_char_length=80, celery_batch_size=1000, from_pretrained='facebook/bart-base', model_type='Twin', nlp=<spacy.lang.en.English object at 0x7f7168e3f160>, asynchr=True, twin_delimiter='###tt9HHSlkWoUM###')

In [6]:
from transformers import __version__
__version__

'4.15.0'

## Extracting examples for LM

In [7]:
from smtag.extract import ExtractorXML

In [13]:
# ! rm -fr /data/text/oapmc_twin
# ! rm -fr /data/text/emboj_twin

In [14]:
corpus = "/data/xml/emboj_all"
text_examples = "/data/text/emboj_twin"
xpath = [".//article-meta/title-group/article-title", ".//abstract"]
sentence_level = False
keep_xml = False
inclusion_probability = 1.0

In [15]:
extractor_lm = ExtractorXML(
    corpus,
    destination_dir=text_examples,
    sentence_level=sentence_level,
    xpath=xpath,
    keep_xml=keep_xml,
    inclusion_probability=inclusion_probability
)

/data/text/emboj_twin created


In [16]:
extractor_lm.extract_from_corpus()

100%|██████████| 4/4 [00:08<00:00,  2.01s/it]


{PosixPath('/data/text/emboj_twin/train.txt'): 7991,
 PosixPath('/data/text/emboj_twin/eval.txt'): 2651,
 PosixPath('/data/text/emboj_twin/test.txt'): 2659}

same via CLI:


```bash
python -m smtag.cli.prepro.extract /data/xml/emboj_all /data/text/emboj_twin --xpath ".//article-meta/title-group/article-title" ".//abstract"
```

```bash
python -m smtag.cli.prepro.extract /data/xml/oapmc_articles /data/text/oapmc_twin --xpath ".//article-meta/title-group/article-title" ".//abstract"
```

```bash
python -m smtag.cli.prepro.extract /data/xml/emboj_all /data/text/emboj_trivial --xpath ".//article-meta/title-group/article-title" ".//article-meta/title-group/article-title"
```

## Preparing tokenized dataset for LM

In [13]:
from smtag.dataprep import PreparatorLM

In [ ]:
tokenized_examples = "/data/json/emboj_twin"

In [15]:
# ! rm -fr /data/json/oapmc_twin
! rm -fr /data/json/emboj_twin

In [16]:
prep_lm = PreparatorLM(
    text_examples,
    tokenized_examples,
    max_length=config.max_length
)

/data/json/emboj_twin created


In [17]:
prep_lm.run()

Preparing: train


100%|██████████| 7991/7991 [00:16<00:00, 488.85it/s]


Preparing: eval


100%|██████████| 2651/2651 [00:04<00:00, 605.43it/s]


Preparing: test


100%|██████████| 2659/2659 [00:04<00:00, 612.81it/s]


same vie CLI:
    
```bash
python -m smtag.cli.lm.dataprep /data/text/oapmc_twin /data/json/oapmc_twin
```

```bash
python -m smtag.cli.lm.dataprep /data/text/emboj_twin /data/json/emboj_twin
```

```bash
python -m smtag.cli.lm.dataprep /data/text/emboj_trivial /data/json/emboj_trivial
```

## Train LM

In [74]:
%autoreload 2

In [75]:
from smtag.train.train_lm import (
    train as train_lm,
    TrainingArgumentsLM
)

In [76]:
no_cache = False  # ALWAYS CHECK THIS IN CASE OF PROBLEMS!!!
loader_path = "./smtag/loader/loader_twin.py"
data_config_name = "NOLM"  # "SEQ2SEQ" ot have also a language model
tokenizer = config.tokenizer  # tokenizer has to be the same application-wide
model_type = "Twin"
from_pretrained = config.from_pretrained

In [77]:
tokenizer.mask_token = '<mask>'  # why is this here? maybe because in case of character-level tokenizer
# tokenizer.unk_token = '<unk>'

In [78]:
training_args_tokcl = TrainingArgumentsLM(
    num_train_epochs = 3,
    logging_steps = 10,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
)
training_args_tokcl

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


TrainingArgumentsLM(output_dir='/lm_models', overwrite_output_dir=True, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=True, per_device_train_batch_size=4, per_device_eval_batch_size=4, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, log_level=-1, log_level_replica=-1, log_on_each_node=True, logging_dir='/lm_models/runs/Apr12_13-42-48_ea25f0d9a7eb', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=100, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit=5, save_on_each_node=False, no_cuda=False, seed=42, bf16=

In [79]:
train_lm(
    training_args_tokcl,
    loader_path,
    data_config_name,
    tokenized_examples,
    no_cache,
    tokenizer,
    model_type,
    from_pretrained
)

tokenizer vocab size: 50265

Loading datasets found in /data/json/emboj_twin.
using ./smtag/loader/loader_twin.py as dataset loader.


  0%|          | 0/3 [00:00<?, ?it/s]


Training with 7991 examples.
Evaluating on 2651 examples.


loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.a243ed957122436adb0b8d8e9d20f896f45c174b6324d625ca0a20a84f72a910
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id":


Training arguments:
TrainingArgumentsLM(
_n_gpu=4,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=100,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_di

...



 sinks[<s>] performed[ZE] reunited[B] reunited[,]td[a] Equip[verte]appiness[brate]appiness[hom]appiness[olog]appiness[of] abolition[D]td[ros]SPONSORED[oph]appiness[ila]appiness[Z] Radiant[f] compan[h] writer[-]appiness[1]appiness[,]td[is]appiness[a] grandparents[negative] compan[regulator]appiness[of]eu[muscle]appiness[differentiation]appiness[.]appiness[</s>]




 panels[<s>]limited[A]emption[number]idge[of]idge[genes]emption[,]emption[spanning]emption[the] slee[evolutionary]idge[scale]emption[from] slee[yeast] slee[to] slee[mammals] seated[,] slee[that] seated[are]emption[involved]emption[in]emption[spatial]idge[and]idge[temporal] Catholics[pattern]emption[ing]emption[during]emption[development] slee[contain] slee[zinc]emption[finger]emption[and]idge[home]emption[od]emption[om]emption[ain]emption[motif]emption[s]emption[.]emption[One] seated[such]emption[zinc]idge[finger]emption[/]emption[home] seated[od]emption[om]emption[ain]emption[protein]emption[is]emption[D]emption[ros]emptio

INFO:***** Running Evaluation *****
INFO:  Num examples = 2651
INFO:  Batch size = 16




 revival[<s>] Replace[Mess]SPONSORED[ages]near[from]SPONSORED[forgotten] contraction[friends]eu[:]eu[classic]eu[cell]SPONSORED[ad]SPONSORED[hesion] contraction[molecules] contraction[inhibit] contraction[regeneration] contraction[too] contraction[</s>]




 extinction[<s>]yle[A] seated[necessary] seated[step] seated[toward] entrepreneurs[complete] entrepreneurs[functional] entrepreneurs[recovery] seated[after] entrepreneurs[spinal] seated[cord] seated[injury] seated[is]emption[the] Governors[regeneration] seated[of] seated[ax] seated[ons] seated[.]emption[Ax] seated[on] seated[reg]emption[rowth] seated[after] entrepreneurs[injury] seated[is]emption[prevented]emption[by] seated[a] entrepreneurs[myriad] entrepreneurs[of] entrepreneurs[intrinsic] Veteran[and] Veteran[extr]emption[ins]emption[ic] Veteran[factors] trou[.]emption[In] entrepreneurs[this] entrepreneurs[issue] seated[of] Governors[The] Governors[EM]emption[BO]emption[Journal]emption[,] seated[Huang] seated[et]emption[]emption

INFO:***** Running Evaluation *****
INFO:  Num examples = 2651
INFO:  Batch size = 16


KeyboardInterrupt: 

#### With CLI:

```bash
python -m smtag.cli.lm.train smtag/loader/loader_twin.py SEQ2SEQ --data_dir /data/json/oapmc_twin --per_device_train_batch_size=6 --per_device_eval_batch_size=6 --logging_steps=4000 --num_train_epochs=1 --no_cache --save_steps=12000
```

```bash
python -m smtag.cli.lm.train smtag/loader/loader_twin.py SEQ2SEQ --data_dir /data/json/emboj_twin --per_device_train_batch_size=4 --per_device_eval_batch_size=4 --logging_steps=100 --num_train_epochs=1 --no_cache 
```

```bash
python -m smtag.cli.lm.train smtag/loader/loader_twin.py SEQ2SEQ --data_dir /data/json/emboj_trivial --per_device_train_batch_size=4 --per_device_eval_batch_size=4 --logging_steps=100 --num_train_epochs=1 --no_cache 
```

```bash
python -m smtag.cli.lm.train smtag/loader/loader_twin.py NOLM --data_dir /data/json/oapmc_twin --per_device_train_batch_size=64 --per_device_eval_batch_size=64 --logging_steps=50 --num_train_epochs=100   --save_steps=12000
```